want to roll a FWTD. Will stick a FWTD at both ends of a rigid beam that can rotate freely

### Preamble

In [1]:
import sympy as sym
import sympy.physics.mechanics as me
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import odeint,LSODA,BDF,solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '..\..'))
import sympyTransforms as symt
import custom_plot_objects as cpo
import FwtModels.dof2 as dof2
import FwtModels.AeroModels as ams

me.mechanics_printing()

### Define the Model

#### Create Parameter instance

In [2]:
half = sym.Rational(1,2)

p = dof2.FwtParameters(3)

# fwt params
p.Lambda = dof2.FwtVariable(np.deg2rad(10),'Lambda') # flare angle
p.y_f = dof2.FwtVariable(0,'y_f') # spanwise location
    
# Attitude Parmas
p.rho = dof2.FwtVariable(1.225,'rho') # density
p.V = dof2.FwtVariable(10,'V') # velocity
p.g  = dof2.FwtVariable(9.81,'g') # gravity
p.alpha_r = dof2.FwtVariable(np.deg2rad(3),'alpha_r') # root AoA

# Main Wing Parameter
p.a_w = dof2.FwtVariable(2*np.pi,'a_w') # C_L of wing
p.k_w = dof2.FwtVariable(0,'k_w') # spring constant fro wing
p.y_w = dof2.FwtVariable(0,'y_w') # spanwise location


# Create some global parameters
p.c = dof2.FwtVariable(0.15,'c') # chord of wing
p.m_t = dof2.FwtVariable(30,'m_t') # wing total mass
p.s_t = dof2.FwtVariable(10,'s_t') # wing total span
p.sigma = dof2.FwtVariable(0.2,'sigma') # FWT as percentage of total span
p.f = dof2.FwtVariable(2,'f') # initial freqency


# Set dependent expressions
p.m_w = p.m_t*(sym.Integer(1)-p.sigma) # mass of inner wing
p.s_w = p.s_t*(sym.Integer(1)-p.sigma) # span of main wing
p.m_f = p.m_t*p.sigma*half # mass of FWT
p.s_f = p.s_t*p.sigma*half # span of each FWT

p.a_f = p.a_w - p.a_w*p.y_f/p.s_f

p.I_xx_f = sym.Rational(1,12)*p.m_f*p.s_f**2 # inertia of FWT (uniform bar)
p.I_xx_w = sym.Rational(1,12)*p.m_w*p.s_w**2 # inertia of wing (uniform bar)
p.l_f = p.s_f*half # location of FWT CoM from Hinge
p.l_w = p.s_w*half # location of main wing CoM

p.k_w = (p.f*2*sym.pi)**2*(p.m_t) # set inner wing Freq to inner_freq

#### Create Transforms

system has 3 degrees of freedom:

q[0] - rotation about the base of the spring (roll)
q[1] - plunging motion
q[2] - FWT fold angle



In [3]:
wing_frame = symt.HomogenousTransform().R_x(p.q[0]).Translate(0,0,p.q[1]).R_y(p.alpha_r)
wing_com_frame = wing_frame.Translate(0,p.l_w,0)

fwt_frame = wing_frame.Translate(0,p.s_w,0).R_x(p.q[2])  # RHS Fwt Frame
fwt_com_frame = fwt_frame.Translate(0,p.l_f,0)


rot = sym.Matrix([p.q[0]+p.q[2],p.alpha_r,0])
wing_rot = sym.Matrix([p.q[0],p.alpha_r,0])


M_fwt = dof2.MassMatrix(p.m_f,I_xx = p.I_xx_f)
M_w = dof2.MassMatrix(p.m_w,I_xx = p.I_xx_w)

bar_fwt = dof2.RigidElement(fwt_com_frame,rot,M_fwt)
bar_wing = dof2.RigidElement(wing_com_frame,wing_rot,M_w)

# Gravitational Forces
fwt_GravityForces = ams.GravityModel(p,fwt_com_frame,sym.Matrix([0,0,p.g*p.m_f]))
wing_GravityForces = ams.GravityModel(p,wing_com_frame,sym.Matrix([0,0,p.g*p.m_w]))

#Aero Forces
fwt_frame_aero = fwt_frame.Translate(0,p.y_f,0)
fwt_AeroForces = ams.AeroModelv2(p,fwt_frame_aero,p.a_f,p.y_f,p.s_f,ams.FwtAoA(p,p.q[2]))

wing_frame_aero = wing_frame.Translate(0,p.y_w,0)
wing_AeroForces = ams.AeroModelv2(p,wing_frame_aero,p.a_w,p.y_w,p.s_w)

#compistie force
CompositeForce = ams.CompositeForce([fwt_GravityForces,wing_GravityForces,fwt_AeroForces,wing_AeroForces])

# potential energy
U = half*p.k_w*(p.q[1])**2

# create instance of the model
sm = dof2.SymbolicModel(U,[M_w,M_fwt],p,ExtForces = CompositeForce)

AttributeError: MutableDenseMatrix has no attribute CalcKE.

## create masses 

In [29]:
y_t = sym.Symbol('y_t')

fwt_frame.Translate(0,y_t,0).BodyVelocity()[2]

In [35]:
p.__dict__['Lambda'].value

In [16]:
p = dof2.FwtParameters(3)
t = me.dynamicsymbols._t
Transform = symt.HomogenousTransform().R_x(p.q[0]).Translate(0,p.q[1],0)

body_v = sym.eye(4)
body_v[:3,:3] = Transform.R.T*Transform.R.diff(t)
body_v[:3,3] = Transform.R.T*Transform.t

sym.simplify(Transform.R.T*Transform.t.diff(t))

⎡  0  ⎤
⎢     ⎥
⎢ q₁̇  ⎥
⎢     ⎥
⎣q₁⋅q₀̇⎦

In [19]:
sym.simplify(Transform.BodyVelocity())

⎡  0  ⎤
⎢     ⎥
⎢ q₁̇  ⎥
⎢     ⎥
⎢q₁⋅q₀̇⎥
⎢     ⎥
⎢ q₀̇  ⎥
⎢     ⎥
⎢  0  ⎥
⎢     ⎥
⎣  0  ⎦

In [6]:
V = sym.ones(6,1)
V[:3,0]

⎡1⎤
⎢ ⎥
⎢1⎥
⎢ ⎥
⎣1⎦

In [5]:
Transform.R.T*Transform.R

⎡1           0                    0         ⎤
⎢                                           ⎥
⎢      2          2                         ⎥
⎢0  sin (q₀) + cos (q₀)           0         ⎥
⎢                                           ⎥
⎢                           2          2    ⎥
⎣0           0           sin (q₀) + cos (q₀)⎦